In [1]:
from __future__ import annotations
import os, csv, random, string
from dataclasses import dataclass, asdict
from datetime import datetime
from typing import List

# ----------------------------
# Configuración / constantes
# ----------------------------
DATA_DIR = os.path.join("data")
CSV_PATH = os.path.join(DATA_DIR, "products.csv")

ALLOWED_CATEGORIES = [
    "Chocolates", "Caramelos", "Mashmelos", "Galletas", "Salados", "Gomas de mascar"
]

# ----------------------------
# Excepciones específicas
# ----------------------------
class ValidationError(Exception):
    """Error genérico de validación."""

class PriceParseError(ValidationError):
    """El precio no pudo convertirse a decimal válido."""

# ----------------------------
# Modelo de datos
# ----------------------------
@dataclass
class Producto:
    nombre: str
    precio: float
    categorias: List[str]
    en_venta: bool
    ts: str  # timestamp ISO para trazabilidad

# ----------------------------
# Utilidades de validación
# ----------------------------
def validar_nombre(nombre: str) -> str:
    if not isinstance(nombre, str) or len(nombre.strip()) == 0:
        raise ValidationError("El nombre no puede estar vacío.")
    if len(nombre.strip()) > 20:
        raise ValidationError("El nombre del producto no debe superar 20 caracteres.")
    return nombre.strip()

def parsear_y_validar_precio(precio_raw: str | float | int) -> float:
    try:
        precio = float(precio_raw)
    except Exception as _:
        # Requisito 8: mensaje específico si escriben texto en precio
        raise PriceParseError("Por favor verifique el campo del precio.")
    if not (0 < precio < 999):
        raise ValidationError("El precio debe ser mayor a 0 y menor a 999.")
    # 2 decimales
    return round(precio, 2)

def validar_categorias(categorias: List[str]) -> List[str]:
    if not categorias:
        raise ValidationError("Debe elegir al menos una categoría.")
    allowed = set(ALLOWED_CATEGORIES)
    cats_limpias = []
    for c in categorias:
        c_ok = str(c).strip()
        if c_ok not in allowed:
            raise ValidationError(f"Categoría inválida: '{c_ok}'.")
        cats_limpias.append(c_ok)
    # único y ordenado (opcional)
    return sorted(list(set(cats_limpias)))

def validar_en_venta(valor) -> bool:
    # Acepta radio Si/No o booleanos equivalentes
    mapping_true = {"si", "s", "true", "1", True, 1}
    mapping_false = {"no", "n", "false", "0", False, 0}
    v = str(valor).strip().lower() if not isinstance(valor, bool) else valor
    if v in mapping_true:
        return True
    if v in mapping_false:
        return False
    raise ValidationError("Valor inválido para '¿está en venta?'. Use Sí/No.")

# ----------------------------
# Persistencia CSV
# ----------------------------
def asegurar_directorio():
    os.makedirs(DATA_DIR, exist_ok=True)

def guardar_csv(productos: List[Producto]) -> None:
    asegurar_directorio()
    write_header = not os.path.exists(CSV_PATH)
    with open(CSV_PATH, "a", newline="", encoding="utf-8") as f:
        writer = csv.DictWriter(
            f,
            fieldnames=["nombre", "precio", "categorias", "en_venta", "ts"]
        )
        if write_header:
            writer.writeheader()
        for p in productos:
            row = asdict(p)
            # guarda categorías como lista separada por ';'
            row["categorias"] = ";".join(p.categorias)
            writer.writerow(row)

# ----------------------------
# Generación sintética
# ----------------------------
ADJETIVOS = ["Dulce", "Crujiente", "Suave", "Clásico", "Premium", "Mini", "Maxi"]
BASES = ["Bite", "Choco", "Caramel", "Mash", "Cookie", "Salty", "Gum"]

def nombre_sintetico() -> str:
    # Genera un nombre ≤ 20 caracteres
    for _ in range(10):
        a = random.choice(ADJETIVOS)
        b = random.choice(BASES)
        suf = "".join(random.choices(string.ascii_uppercase + string.digits, k=2))
        n = f"{a}{b}{suf}"
        if len(n) <= 20:
            return n
    # fallback
    return "Dulcino-"+str(random.randint(10, 99))

def generar_producto_valido() -> Producto:
    nombre = validar_nombre(nombre_sintetico())
    precio = parsear_y_validar_precio(round(random.uniform(0.2, 998.9), 2))
    # 1 a 3 categorías válidas
    k = random.randint(1, 3)
    categorias = validar_categorias(random.sample(ALLOWED_CATEGORIES, k))
    en_venta = random.choice([True, False])
    return Producto(
        nombre=nombre,
        precio=precio,
        categorias=categorias,
        en_venta=en_venta,
        ts=datetime.utcnow().isoformat(timespec="seconds")+"Z",
    )

def generar_batch(n: int = 200) -> List[Producto]:
    productos = []
    for _ in range(n):
        productos.append(generar_producto_valido())
    return productos

# ----------------------------
# Demo: registro manual (simula formulario CLI)
# ----------------------------
def crear_desde_input() -> None:
    print("— Registro manual de producto —")
    try:
        nombre = validar_nombre(input("Nombre del producto: "))
        precio = parsear_y_validar_precio(input("Precio del producto: "))
        print("Categorías permitidas:", ", ".join(ALLOWED_CATEGORIES))
        cats_raw = input("Categorías (separa por coma): ")
        categorias = validar_categorias([c.strip() for c in cats_raw.split(",") if c.strip()])
        en_venta = validar_en_venta(input("¿Está en venta? (Si/No): "))

        producto = Producto(
            nombre, precio, categorias, en_venta,
            ts=datetime.utcnow().isoformat(timespec="seconds")+"Z"
        )
        guardar_csv([producto])
        print("Felicidades su producto se agregó.")
    except PriceParseError as e:
        print(str(e))  # “Por favor verifique el campo del precio.”
        print("Lo sentimos no pudo crear este producto.")
    except ValidationError as e:
        print("Lo sentimos no pudo crear este producto.")
        print("Detalle:", e)

# ----------------------------
# Main
# ----------------------------
if __name__ == "__main__":
    # 1) Genera lote sintético (modifica n si quieres)
    lote = generar_batch(n=300)
    guardar_csv(lote)
    print(f"✅ Generados y guardados {len(lote)} productos en {CSV_PATH}")

    # 2) (Opcional) permitir 1 registro manual por CLI
    # crear_desde_input()

✅ Generados y guardados 300 productos en data\products.csv


C:\Users\Steph\AppData\Local\Temp\ipykernel_4200\387027979.py:134: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  ts=datetime.utcnow().isoformat(timespec="seconds")+"Z",
